In [94]:
import tensorflow as tf
print(tf.__version__)

import tensorflow_datasets as tfds
from tensorflow.keras.layers import Layer
from tensorflow.keras.models import Model

from keras import backend as K

import numpy as np
import pandas as pd
import os

2.15.0


In [5]:
### Custom Lambda Layers in TensorFlow

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    ## here instead of using 'relu', we provied customized lambda layer.
    tf.keras.layers.Dense(units=128),
    tf.keras.layers.Lambda(lambda x: tf.abs(x)),
    tf.keras.layers.Dense(units=10, activation='softmax')
])


In [8]:
### Custom function for lambda layers.

## Let's give customize the relu in a function, and pass it in lambda layer.

def my_relu(x):
  return K.maximum(0.5, x)

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    ## here instead of using 'relu', we provied customized lambda layer.
    tf.keras.layers.Dense(units=128),
    tf.keras.layers.Lambda(my_relu),
    tf.keras.layers.Dense(units=10, activation='softmax')
])

### 1) Implementing custom layers.

In [65]:
### Implementing custom layers.
### A layer in tensorflow is a class that collects parameters to encapsulate state(weights) and computation(forward pass). Here weights can be trainable or set to be un-trainable.


class SimpleDense(Layer): ## inherit tensorflow's Layer class.

  def __init__(self, units=32, activation=None):
    super(SimpleDense, self).__init__()
    self.units = units
    self.activation = tf.keras.activations.get(activation)

  def build(self, input_shape): # create/initialize a state of the layer(defining weights, kernel(w) and bias(b))

    w_init = tf.random_normal_initializer()

    self.w = tf.Variable(name = 'kernel',
                         initial_value = w_init(shape=(input_shape[-1], self.units),
                                                dtype='float32'
                                                ),
                         trainable=True
                         )


    b_init = tf.zeros_initializer()

    self.b = tf.Variable(name = 'bias',
                         initial_value=b_init(shape=(self.units,),
                                              dtype='float32'
                                              ),
                         trainable=True
                         )

  def call(self, inputs): ## defines the computation in the layers from inputs to outputs
    return self.activation( tf.matmul(inputs, self.w) + self.b )



In [66]:
my_dense = SimpleDense(units=1)
x = tf.ones((1,1))
print(x)
y = my_dense(x)

print(my_dense.variables)

tf.Tensor([[1.]], shape=(1, 1), dtype=float32)
[<tf.Variable 'simple_dense_33/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[0.02248342]], dtype=float32)>, <tf.Variable 'simple_dense_33/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


In [76]:
### Training a neural network with your custom Layes.

xs = np.reshape(np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float), newshape=(6,1))
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)



my_layer = SimpleDense(units=1)

model = tf.keras.Sequential([my_layer])

model.compile(loss='mean_squared_error', optimizer='sgd')

model.fit(xs, ys, epochs=500, verbose=0)

model.predict([10.0])

print(my_layer.weights)

1/1 [==============================] - 0s 54ms/step
[<tf.Variable 'simple_dense_41/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[1.9972883]], dtype=float32)>, <tf.Variable 'simple_dense_41/bias:0' shape=(1,) dtype=float32, numpy=array([-0.99159294], dtype=float32)>]


### 2) Complex Architectures with the Functional API - Custom Models

In [78]:
### Custom Models

class WideAndDeepModel(Model):

  def __init__(self, units=30, activation='relu', **kwargs):
    super().__init__(**kwargs)
    self.hidden_1 = tf.keras.layers.Dense(units = units, activation = activation)
    self.hidden_2 = tf.keras.layers.Dense(units = units, activation = activation)
    self.main_output = tf.keras.layers.Dense(units = 1)
    self.aux_output = tf.keras.layers.Dense(units = 1)


  def call(self, inputs):

    input_a, input_b = inputs

    hidden_1 = self.hidden_1(input_b)
    hidden_2 = self.hidden_2(hidden_1)
    concat = tf.keras.layers.concatenate([input_a, hidden_2])
    main_output = self.main_output(concat)
    aux_output = self.aux_output(hidden_2)

    return main_output, aux_output


In [79]:
model = WideAndDeepModel()

### Using Model class to simplify complex architectures - Residual Networks (ResNets)

In [83]:
## convolutional layers block
class CNNResidual(Layer):

  def __init__(self, layers, filters, **kwargs):
    super().__init__(**kwargs)
    self.hidden = [ tf.keras.layers.Conv2D(filters, kernel_size=(3,3), activation='relu')
                  for _ in range(layers) ]


  def call(self, inputs):
    x = inputs
    for layer in self.hidden:
      x = layer(x)
      return inputs + x

## Dense layers block
class DNNResidual(Layer):

  def __init__(self, layers, neurons, **kwargs):
    super().__init__(**kwargs)
    self.hidden = [ tf.keras.layers.Dense(units = neurons, activation='relu')
                  for _ in range(layers) ]


  def call(self, inputs):
    x = inputs
    for layer in self.hidden:
      x = layer(x)
      return inputs + x

## create residual network from convolutional and dense layer blocks.
class MyResidual(Model):
  def __init__(self, **kwargs):
    self.hidden_1 = tf.keras.layers.Dense(units=30, activation='relu')
    self.blocl_1 = CNNResidual(2, 32).   ## convolutional block
    self.blocl_2 = DNNResidual(2, 64)    ## dense block
    self.out = tf.keras.layers.Dense(units=1)


  def call(self, inputs):
    x = self.hidden_1(inputs)
    x = self.blocl_1(x)

    for _ in range(1, 4):
      x = self.blocl_2(x)

    return self.out(x)

### ResNet-18 architechture

In [101]:
## build indentity block

class IdentityBlock(tf.keras.Model):
  def __init__(self, filters, kernel_size):
    super(IdentityBlock, self).__init__(name='')

    self.conv1 = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')
    self.bn1 = tf.keras.layers.BatchNormalization()

    self.conv2 = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')
    self.bn2 = tf.keras.layers.BatchNormalization()

    self.act = tf.keras.layers.Activation('relu')
    self.add = tf.keras.layers.Add()

  def call(self, input_tensor):
    x = self.conv1(input_tensor)
    x = self.bn1(x)
    x = self.act(x)

    x = self.conv2(x)
    x = self.bn2(x)

    x = self.add([x, input_tensor])
    x = self.act(x)

    return x





In [102]:
class ResNet(tf.keras.Model):

  def __init__(self, num_classes): ## 'num_classes' for classifier layer
    super(ResNet, self).__init__()

    self.conv = tf.keras.layers.Conv2D(64, 7, padding='same')
    self.bn = tf.keras.layers.BatchNormalization()
    self.act = tf.keras.layers.Activation('relu')
    self.max_pool = tf.keras.layers.MaxPool2D(pool_size=(3,3))

    self.id1a = IdentityBlock(64, 3)
    self.id1b = IdentityBlock(64, 3)

    self.global_pool = tf.keras.layers.GlobalAveragePooling2D()
    self.classifier = tf.keras.layers.Dense(units = num_classes, activation='softmax')

  def call(self, inputs):
    x = self.conv(inputs)
    x = self.bn(x)
    x = self.act(x)
    x = self.max_pool(x)

    x = self.id1a(x)
    x = self.id1b(x)

    x = self.global_pool(x)

    return self.classifier(x)


In [103]:

def preprocess(features):
  return tf.cast(features['image'], tf.float32) /255.0 , features['label']

resnet = ResNet(10)
resnet.compile(optimizer='adam',
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy']
               )

dataset = tfds.load('mnist',  split='train')
dataset = dataset.map(preprocess).batch(32)

resnet.fit(dataset, epochs=1)


1875/1875 [==============================] - 233s 123ms/step - loss: 0.1245 - accuracy: 0.9665
